In [5]:
from tree import Tree, lemma
import pickle

PATH = 'taxonomy/tree_0.pkl'

In [12]:
def construct_wordmap(wordmap, tree, visited):
    if tree in visited:
        return
    visited.add(tree)
    for c in tree.children:
        construct_wordmap(wordmap, c, visited)
    for syn in tree.synonyms:
        wordmap[syn] = tree


# create lower case words too
def reduced(word):
    res = lemma(word.lower().replace(' ', ''))
    if type(res) == tuple:
        joined = ''.join(list(res))
    if type(res) == list:
        joined = ''.join(res)
    return joined


def _merge_two_nodes(source, destination):
    # check it does not create self loop
    if source == destination:
        return

    # check that it does not create a loop
    destination_children = destination.list_all_childs()
    if source in destination_children:
        return

    for syn in source.synonyms:
        if syn not in destination.synonyms:
            destination.synonyms.append(syn)
    for child in source.children:
        if child not in destination.children:
            destination.add_child(child)


def get_all_appearing_terms(tree, ctx):
    res = []
    for c in tree.children:
        res += get_all_appearing_terms(c, ctx)
    if ctx is None:
        res += [tree.synonyms[0]]
    else:
        found = []
        for syn in tree.synonyms:
            if syn in ctx:
                found.append(syn)
        if len(found) == 0:
            res += [tree.synonyms[0]]
        else:
            res += found
    return res


def _get_min_number_steps(source_tree, target_tree):
    if source_tree == target_tree:
        return 0
    min_steps = 1000000
    for c in source_tree.children:
        min_steps = min(min_steps, _get_min_number_steps(c, target_tree))
    return 1 + min_steps


def _get_num_edges(tree):
    if len(tree.children) == 0:
        return 0
    num_edges = 0
    for c in tree.children:
        num_edges += 1 + _get_num_edges(c)
    return num_edges


def compute_general_metrics(tree):
    nodes = set(tree.get_nodes_list())
    sum_num_children = 0
    for n in nodes:
        sum_num_children += len(set(n.children))
    avg_children = float(sum_num_children) / len(nodes)

    max_num_children = 0
    min_num_children = 1000000
    for n in nodes:
        max_num_children = max(max_num_children, len(set(n.children)))
        min_num_children = min(min_num_children, len(set(n.children)))

    depth = tree.get_depth()

    avg_steps_to_leaf = 0
    for n in nodes:
        avg_steps_to_leaf += _get_min_number_steps(tree, n)
    avg_steps_to_leaf = float(avg_steps_to_leaf) / len(nodes)

    num_nodes = len(nodes)
    num_leafs = len(set(tree.get_leaf_nodes()))
    num_edges = _get_num_edges(tree)

    print("Average number of children: ", avg_children)
    print("Max number of children: ", max_num_children)
    print("Min number of children: ", min_num_children)
    print("Depth: ", depth)
    print("Average steps to leaf: ", avg_steps_to_leaf)
    print("Number of nodes: ", num_nodes)
    print("Number of leafs: ", num_leafs)
    print("Number of edges: ", num_edges)


In [13]:
with open(PATH, 'rb') as f:
    reconstructed_tree = pickle.load(f)

wordmap = {}
construct_wordmap(wordmap, reconstructed_tree, set())

keys_list = list(wordmap.keys())
for word in keys_list:
    if reduced(word) in wordmap:
        _merge_two_nodes(wordmap[word], wordmap[reduced(word)])
    wordmap[reduced(word)] = wordmap[word]
    wordmap[reduced(word)].synonyms.append(reduced(word))

In [14]:
def _quick_fix_tree(tree, visited, level, path):
    if tree in visited:
        raise Exception("Loop detected")
    visited.add(tree)
    to_remove = []
    for c in tree.children:
        try:
            _quick_fix_tree(c, visited, level + 1, path + [tree.synonyms[0]])
        except:
            to_remove.append(c)
    for c in to_remove:
        tree.children.remove(c)
    visited.remove(tree)
    for syn in tree.synonyms:
        wordmap[syn] = tree

# tree should have loops, but it might be necessary to remove them in case there are some
visited = set()
_quick_fix_tree(reconstructed_tree, visited, 0, [])

In [15]:
compute_general_metrics(reconstructed_tree)

Average number of children:  1.0
Max number of children:  17
Min number of children:  0
Depth:  4
Average steps to leaf:  1.9833333333333334
Number of nodes:  60
Number of leafs:  53
Number of edges:  60
